In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

In [2]:
DATA_PATH = r"C:\Users\sahab\Downloads\Telegram Desktop\data_ts.csv"

In [3]:
data = pd.read_csv(DATA_PATH)\
    .drop(columns=["Unnamed: 0"])

In [4]:
data_df_columns = list(pd.read_csv(
    r"C:\Users\sahab\Downloads\Telegram Desktop\zalupa.csv"
).drop(
    columns=["Unnamed: 0", "label", "nan_num"]
).columns) + ["tractor"]

In [5]:
data = data[data_df_columns]

data.to_csv("checkpoint_1.csv")

In [7]:
data_size_before = len(data)

data = data.drop_duplicates()
nan_count_series = data.isnull().sum(axis=1)

data["nun_cnt"] = nan_count_series
data = data[data["nun_cnt"] <= len(data.columns) - 2]

data_size_after = len(data)

data_size_before, data_size_after

(3156090, 3105784)

In [8]:
data = data.fillna(method = 'ffill', axis = 0)
data = data.fillna(0, axis = 0)

C:\Users\sahab\AppData\Local\Temp\ipykernel_13016\919308821.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method = 'ffill', axis = 0)


In [9]:
data_index = data[
    [
        "tractor",
        "Дата и время"
    ]
]

data = data.drop(
    columns=[
        "Дата и время",
        "tractor",
        "nun_cnt"
    ]
)

In [10]:
clf = IsolationForest(
    random_state=42
).fit(data)

In [11]:
i_forest_predict = clf.predict(data)
pd.Series(i_forest_predict).value_counts()

 1    2545839
-1     559945
Name: count, dtype: int64

In [12]:
data["label"] = i_forest_predict
data["label"] = data["label"].apply(
    lambda x: 0 if x == 1 else 1
)

In [17]:
data_concated = pd.concat(
    [data, data_index],
    axis=1
)

In [20]:
data_anomaly_for_all_timestamps = data_concated.groupby(
    "tractor"
).agg({"label": "sum"})

data_all_timestamps = data_concated.groupby(
    "tractor"
).agg({"label": "count"})

In [27]:
data_timestaps_merged = data_anomaly_for_all_timestamps.merge(
    data_all_timestamps, on="tractor"
)

data_timestaps_merged.columns = [
    "anomaly_timestamps_cnt",
    "all_timestamps_cnt"
]

len(data_timestaps_merged.index)

14

In [28]:
data_timestaps_merged["anomaly_ratio"] = data_timestaps_merged["anomaly_timestamps_cnt"] /\
    data_timestaps_merged["all_timestamps_cnt"]

In [35]:
data_timestaps_merged = data_timestaps_merged.reset_index()
data_timestaps_merged

,tractor,anomaly_timestamps_cnt,all_timestamps_cnt,anomaly_ratio
0,022C4097,12471,90392,0.137966
1,023T0569,33829,86165,0.392607
2,023Т0049,31956,92842,0.344198
3,023Т0255,6239,53621,0.116354
4,023Т0719,25133,72498,0.346672
5,C1969,23660,175054,0.135158
6,Р3106,34511,101700,0.339341
7,С0790,62177,279786,0.222231
8,С0873,1331,19018,0.069986
9,С1067,45472,841694,0.054024


In [40]:
answer_list = []

for example in data_timestaps_merged.to_dict("records"):
    answer_list.append(
        {
            "id": example["tractor"],
            "anomaly_timestamps_cnt": example["anomaly_timestamps_cnt"],
            "anomaly_ratio": example["anomaly_ratio"]
        }
    )

In [41]:
answer_list

[{'id': '022C4097',
  'anomaly_timestamps_cnt': 12471,
  'anomaly_ratio': 0.1379657491813435},
 {'id': '023T0569',
  'anomaly_timestamps_cnt': 33829,
  'anomaly_ratio': 0.3926072071026519},
 {'id': '023Т0049',
  'anomaly_timestamps_cnt': 31956,
  'anomaly_ratio': 0.34419766915835504},
 {'id': '023Т0255',
  'anomaly_timestamps_cnt': 6239,
  'anomaly_ratio': 0.11635366740642658},
 {'id': '023Т0719',
  'anomaly_timestamps_cnt': 25133,
  'anomaly_ratio': 0.3466716323208916},
 {'id': 'C1969',
  'anomaly_timestamps_cnt': 23660,
  'anomaly_ratio': 0.1351582940121334},
 {'id': 'Р3106',
  'anomaly_timestamps_cnt': 34511,
  'anomaly_ratio': 0.33934119960668635},
 {'id': 'С0790',
  'anomaly_timestamps_cnt': 62177,
  'anomaly_ratio': 0.22223056192947466},
 {'id': 'С0873',
  'anomaly_timestamps_cnt': 1331,
  'anomaly_ratio': 0.06998632874119255},
 {'id': 'С1067',
  'anomaly_timestamps_cnt': 45472,
  'anomaly_ratio': 0.054024384158613464},
 {'id': 'С1178',
  'anomaly_timestamps_cnt': 6386,
  'anomal

In [43]:
data_concated[data_concated["tractor"] == "С2177"]

,"Давл.масла двиг.,кПа",КПП. Температура масла,КПП. Давление масла в системе смазки,Скорость,ДВС. Давление смазки,ДВС. Температура охлаждающей жидкости,"Давление в пневмостистеме (spn46), кПа",Электросистема. Напряжение,ДВС. Частота вращения коленчатого вала,Нейтраль КПП (spn3843),...,Засоренность фильтра ДВС (spn3845),Засоренность фильтра рулевого управления (spn3844),Засоренность фильтра навесного оборудования (spn3851),Аварийная температура масла в гидросистеме (spn3849),Аварийное давление в I контуре тормозной системы (spn3848),Аварийное давление в II контуре тормозной системы (spn3855),Зарядка АКБ (spn3854),label,tractor,Дата и время
2034455,160.0,-273.0,928.0,0.0,160.0,69.0,816.0,28.20,699.625,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,С2177,2023-01-05 00:00:09
2034456,160.0,-273.0,928.0,0.0,160.0,68.0,816.0,14.10,700.125,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,С2177,2023-01-05 00:00:39
2034457,172.0,-273.0,928.0,0.0,172.0,68.0,816.0,28.20,701.125,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,С2177,2023-01-05 00:01:09
2034458,172.0,-273.0,928.0,0.0,172.0,68.0,816.0,28.20,699.000,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,С2177,2023-01-05 00:01:39
2034459,172.0,-273.0,928.0,0.0,172.0,68.0,816.0,14.20,701.000,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,С2177,2023-01-05 00:02:09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2285613,128.0,-273.0,1024.0,0.0,128.0,79.0,816.0,14.10,699.625,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,С2177,2023-01-07 23:56:06
2285614,140.0,-273.0,1040.0,0.0,140.0,79.0,816.0,14.05,699.000,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,С2177,2023-01-07 23:57:36
2285615,140.0,-273.0,1040.0,0.0,140.0,79.0,816.0,14.05,701.375,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,С2177,2023-01-07 23:57:06
2285616,140.0,-273.0,1040.0,0.0,140.0,79.0,816.0,28.00,700.250,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1,С2177,2023-01-07 23:58:36
